## Importar librerias a utilizar

In [1]:
import pandas as pd

## Lectura de los datos

In [14]:
# Lectura de sitios dentro de Louisiana e Indiana (Google)
sitios= pd.read_parquet("../../Datasets/final-ds/sitios_combined_full") #Obtenido del archivo ETL/business_Google.ipynb

# Reviews Indiana
indiana= pd.read_parquet("../../Datasets/merge/ReviewIndiana")  # Obtenido del archivo ETL/estados

# Reviews Louisiana
louisiana= pd.read_parquet("../../Datasets/merge/ReviewLouisiana") # Obtenido del archivo ETL/estados

- Realizamos la union de los sitios con los estados que correspondan

In [15]:
indiana_sitios = indiana.merge(sitios,how="inner",on="gmap_id")
louisiana_sitios = louisiana.merge(sitios,how="inner",on="gmap_id")

- Observamos la cantidad de registros que quedaron en cada estado

In [6]:
louisiana_sitios.shape, indiana_sitios.shape

((609940, 15), (1126351, 15))

- Chequeo de valores nulos

In [16]:
louisiana_sitios.isna().sum()

user_id                  0
rating                   0
text                280034
gmap_id                  0
date                     0
name                     0
address               1260
latitude                 0
longitude                0
category               202
avg_rating               0
num_of_reviews           0
MISC                 32058
relative_results     13953
url                      0
dtype: int64

In [17]:
indiana_sitios.isna().sum()

user_id                  0
rating                   0
text                494064
gmap_id                  0
date                     0
name                     0
address               2735
latitude                 0
longitude                0
category               468
avg_rating               0
num_of_reviews           0
MISC                 57426
relative_results     30161
url                      0
dtype: int64

- Rellenamos los valores nulos de la columna `category` y la transformamos en string. Esto nos servira para el Analisis Exploratorio de Datos

In [19]:
louisiana_sitios.fillna({"category":"[]"},inplace=True)
indiana_sitios.fillna({"category":"[]"},inplace=True)

In [20]:
louisiana_sitios["category"] = louisiana_sitios["category"].apply(lambda x: " ".join(x) if len(x)>0 else "")

In [21]:
indiana_sitios["category"] = indiana_sitios["category"].apply(lambda x: " ".join(x) if len(x)>0 else "")

In [22]:
hotel = louisiana_sitios[louisiana_sitios["category"].str.contains("hotel")]

- Filtramos para obtener una categoria principal

In [35]:
def checkCategory(word):
    if "Hotel" in word or "hotel" in word:
        return "Hotel"
    elif "Restaurant" in word or "restaurant" in word:
        return "Restaurant"
    else: return "Otros"
    
louisiana_sitios["main_category"] = louisiana_sitios["category"].apply(checkCategory)
indiana_sitios["main_category"] = louisiana_sitios["category"].apply(checkCategory)

- Exportamos a parquet

In [36]:
indiana_sitios.to_parquet("../../Datasets/merge/indiana_sitios.parquet", engine="pyarrow")
louisiana_sitios.to_parquet("../../Datasets/merge/louisiana_sitios.parquet", engine="pyarrow")